## TODO

- Меньше цветов (меньше расширений, возможно стоит разделить их на kt+py, ipynb+csv, other)
- Переименовать 'extension' во что-то более понятное
- Использовать данные из Google Search Console
- Найти корреляцию между продуктивностью и температурой в Баре или другими данными
- Использовать данные про файлы (еще один датасет)
- Использовать информацию added, changed, fixed, link из releases_df

In [1]:
from datetime import datetime

import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
commits_df = pd.read_csv("data/generated/commits.csv", sep=";", parse_dates=["date"])
commits_df["deletions_neg"] = -commits_df["deletions"]
print(commits_df.shape)
commits_df.head()

(1092, 7)


,hash,author,date,extension,additions,deletions,deletions_neg
0,d711892cbac5c522852df5029a9c091924baedbd,Valentin Dovidaytis,2024-12-13 18:48:41+01:00,kts,1.0,1.0,-1.0
1,d711892cbac5c522852df5029a9c091924baedbd,Valentin Dovidaytis,2024-12-13 18:48:41+01:00,py,1.0,1.0,-1.0
2,dfbd0ca13d1717fbe8f6ab187367efae5c8a2244,Valentin Dovidaytis,2024-12-13 18:47:00+01:00,md,30.0,24.0,-24.0
3,dfbd0ca13d1717fbe8f6ab187367efae5c8a2244,Valentin Dovidaytis,2024-12-13 18:47:00+01:00,js,1.0,1.0,-1.0
4,dfbd0ca13d1717fbe8f6ab187367efae5c8a2244,Valentin Dovidaytis,2024-12-13 18:47:00+01:00,kts,1.0,1.0,-1.0


In [4]:
releases_df = pd.read_csv("data/original/releases.csv", parse_dates=["date"])
print(releases_df.shape)
releases_df.head()

(15, 8)


,hash,version,kind,date,added,changed,fixed,link
0,dfbd0ca13d1717fbe8f6ab187367efae5c8a2244,4.5.2,patch,2024-12-13 18:47:00+01:00,0,2,18,https://github.com/JetBrains/lets-plot/release...
1,55105aaf81cb6f30c7612714110023aa4a8b6c71,4.5.1,patch,2024-10-25 17:25:38+02:00,0,1,2,https://github.com/JetBrains/lets-plot/release...
2,fed606ae898a3f2a3b0649dbdaf943504265eff5,4.5.0,minor,2024-10-23 17:22:18+02:00,9,4,14,https://github.com/JetBrains/lets-plot/release...
3,c244299d36ee5b6544097706fe8e954e64abd9ae,4.5.0rc2,rc,2024-10-18 13:49:34+02:00,0,0,0,https://github.com/JetBrains/lets-plot/release...
4,2142b64bfeed4b0d7946d477b04cf90fa23c28b1,4.5.0rc1,rc,2024-10-17 15:21:10+02:00,0,0,0,https://github.com/JetBrains/lets-plot/release...


In [5]:
def get_dt(year=2024, month=1, day=1):
    return int(round(1_000 * datetime(year, month, day).timestamp()))

def get_tooltip(kind):
    return layer_tooltips().title("@..count.. {0}".format(kind))\
                           .line("@|@extension")\
                           .line("@|@date").format("@date", "%d %b")

filtered_commits_df = commits_df[commits_df["extension"].isin(["kt", "py", "js", "kts", "gradle", "md", "ipynb", "csv"])]
filtered_releases_df = releases_df[releases_df["kind"] != "rc"]

ggplot() + \
    geom_rect(xmin=get_dt(), xmax=get_dt(month=12, day=31), ymin=0, ymax=10**30, fill="green", size=0, alpha=.5) + \
    geom_rect(xmin=get_dt(), xmax=get_dt(month=12, day=31), ymin=-10**30, ymax=0, fill="red", size=0, alpha=.5) + \
    geom_histogram(aes(x="date", y="..count..", weight="additions", paint_a="extension"), \
                   data=filtered_commits_df, bins=61, boundary=get_dt(), fill_by="paint_a", \
                   tooltips=get_tooltip("additions")) + \
    geom_histogram(aes(x="date", y="..count..", weight="deletions_neg", paint_a="extension"), \
                   data=filtered_commits_df, bins=61, boundary=get_dt(), fill_by="paint_a", \
                   tooltips=get_tooltip("deletions")) + \
    geom_point(aes(x="date", paint_b="kind"), \
               y=10**25, data=filtered_releases_df, shape=23, size=6, fill_by="paint_b",
               tooltips=layer_tooltips(["kind", "date"]).title("Release v@version").format("@date", "%d %b, %l:%M %p")) + \
    coord_polar() + \
    scale_x_datetime(breaks=[get_dt(month=month) for month in range(1, 13)], expand=[0, 0], format="%b") + \
    scale_y_continuous(limits=[-10**30, 10**30], expand=[0, 0], trans='symlog') + \
    scale_manual("paint_b", values=["light_cyan", "light_magenta"]) + \
    ggsize(1000, 800) + \
    theme(axis_line_x='blank', axis_ticks_x='blank', axis_title_x='blank', axis_text_y='blank', axis_title_y='blank')